# Demo of PYPIT on GMOS Longslit [v1.1]

In [1]:
# import
from importlib import reload
import os
import glob
import numpy as np

from astropy.io import fits

# A few core routines
from pypit.core import arsetup
from pypit.core import arsort
from pypit import arpixels
from pypit.core import arprocimg
from pypit.core import arwave
from pypit.core import arsave
from pypit import arutils
from pypit import arload

# Classes
from pypit import calibrations
from pypit import fluxspec
from pypit import pypitsetup
from pypit import scienceimage

# Spectrgraph and Settings
from pypit.spectrographs.util import load_spectrograph
from pypit.par import pypitpar

## To play along, you need the Development suite and the $PYPIT_DEV environmental variable pointed at it

In [2]:
os.getenv('PYPIT_DEV')

'/data/Projects/Python/PYPIT-development-suite/'

## Spectrograph + Settings

In [3]:
spectro_name='gemini_gmos_south'

In [4]:
spectrograph = load_spectrograph(spectrograph=spectro_name)

In [5]:
spectrograph

<GeminiGMOSSSpectrograph:  spectrograph=gemini_gmos_south, camera=GMOS-S>

### Settings

In [6]:
calib_par = pypitpar.CalibrationsPar(badpix=False,
                                     biasframe=pypitpar.FrameGroupPar('bias',
                                                                      useframe='overscan'))

In [7]:
calib_par.keys()

['caldir',
 'masters',
 'setup',
 'trim',
 'badpix',
 'biasframe',
 'arcframe',
 'pixelflatframe',
 'traceframe',
 'flatfield',
 'wavelengths',
 'slits',
 'tilts',
 'wavecalib']

## Build the fitstbl

### Files

In [8]:
gemini_gmos_files = glob.glob(os.getenv('PYPIT_DEV')+'RAW_DATA/Gemini_GMOS/B600/S2018*')
gemini_gmos_files.sort()
len(gemini_gmos_files)

19

In [9]:
gemini_gmos_files

['/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0180.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0181.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0182.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0183.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0184.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0185.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0186.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0187.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0188.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0189.fits',
 '/data/Projects/Python/PYPIT-developmen

In [10]:
# restricting to 530nm
gemini_gmos_files_530 = gemini_gmos_files[3:7] + [gemini_gmos_files[-5]]
gemini_gmos_files_530

['/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0183.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0184.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0185.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0186.fits',
 '/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0259.fits']

### Looking at the Headers

In [11]:
hdul = fits.open(gemini_gmos_files_530[2])
hdul[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
INSTRUME= 'GMOS-S  '           / Instrument used to acquire data                
OBJECT  = 'GCALflat'           / Object Name                                    
OBSTYPE = 'FLAT    '           / Observation type                               
OBSCLASS= 'partnerCal'         / Observe class                                  
GEMPRGID= 'GS-2018A-Q-205'     / Gemini programme ID                            
OBSID   = 'GS-2018A-Q-205-32'  / Observation ID / Data label                    
DATALAB = 'GS-2018A-Q-205-32

In [12]:
hdul.info()

Filename: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0185.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     252   ()      
  1               -1 ImageHDU       324   (288, 2112)   int16 (rescales to uint16)   
  2               -1 ImageHDU       288   (288, 2112)   int16 (rescales to uint16)   
  3               -1 ImageHDU       252   (288, 2112)   int16 (rescales to uint16)   
  4               -1 ImageHDU       252   (288, 2112)   int16 (rescales to uint16)   
  5               -1 ImageHDU       216   (288, 2112)   int16 (rescales to uint16)   
  6               -1 ImageHDU       180   (288, 2112)   int16 (rescales to uint16)   
  7               -1 ImageHDU       180   (288, 2112)   int16 (rescales to uint16)   
  8               -1 ImageHDU       144   (288, 2112)   int16 (rescales to uint16)   
  9               -1 ImageHDU       108   (288, 2112)   int16 (rescales to uint16)   
 10        

In [13]:
hdul[1].header

XTENSION= 'IMAGE   '           / IMAGE extension                                
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                  288 / length of data axis 1                          
NAXIS2  =                 2112 / length of data axis 2                          
PCOUNT  =                    0 / required keyword; must = 0                     
GCOUNT  =                    1 / required keyword; must = 1                     
BZERO   =                32768 / offset data range to that of unsigned short    
BSCALE  =                    1 / default scaling factor                         
INHERIT =                    F / inherit the primary header                     
DATATYPE= 'Intensity'          / Type of Data                                   
CTYPE1  = 'RA---TAN'           / R.A. in tangent plane projection               
CRPIX1  =     1579.274096031

### PypitSetup

In [14]:
run_par = pypitpar.RunPar()
reduce_par = pypitpar.ReducePar()

In [15]:
# Init
reload(pypitsetup)
setupc = pypitsetup.PypitSetup(spectrograph, run_par, reduce_par)

In [16]:
fitstbl = setupc.build_fitstbl(gemini_gmos_files_530)

[WARNING] :: BINNING keyword not in header. Setting to None
[WARNING] :: SLITNAME keyword not in header. Setting to None
[WARNING] :: DICHNAME keyword not in header. Setting to None
[INFO]    :: Successfully loaded headers for file:
             /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0183.fits
[WARNING] :: BINNING keyword not in header. Setting to None
[WARNING] :: SLITNAME keyword not in header. Setting to None
[WARNING] :: DICHNAME keyword not in header. Setting to None
[INFO]    :: Successfully loaded headers for file:
             /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0184.fits
[WARNING] :: BINNING keyword not in header. Setting to None
[WARNING] :: SLITNAME keyword not in header. Setting to None
[WARNING] :: DICHNAME keyword not in header. Setting to None
[INFO]    :: Successfully loaded headers for file:
             /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180

In [17]:
fitstbl

directory,filename,utc,target,idname,time,date,ra,dec,airmass,binning,exptime,decker,dichroic,dispname,instrume
str72,str19,str10,str12,str6,float64,str10,float64,float64,float64,str4,float64,str4,str4,str11,str17
/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/,S20180711S0183.fits,06:37:17.2,ESO-606-G036,OBJECT,1399443.4794788086,2018-07-11,333.853125,-19.58527778,1.052,None,75.0,None,None,B600+_G5323,gemini_gmos_south
/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/,S20180711S0184.fits,06:39:02.2,ESO-606-G036,OBJECT,1399443.508718023,2018-07-11,333.853125,-19.58527778,1.05,None,75.0,None,None,B600+_G5323,gemini_gmos_south
/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/,S20180711S0185.fits,06:41:03.2,GCALflat,FLAT,1399443.5423247123,2018-07-11,333.853125,-19.58527778,1.049,None,4.0,None,None,B600+_G5323,gemini_gmos_south
/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/,S20180711S0186.fits,06:41:41.2,CuAr,ARC,1399443.552822083,2018-07-11,333.853125,-19.58527778,1.048,None,20.0,None,None,B600+_G5323,gemini_gmos_south
/data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/,S20180711S0259.fits,10:43:27.7,LTT1788,OBJECT,1399447.5795640445,2018-07-11,57.09420833,-39.14358333,1.178,None,60.0,None,None,B600+_G5323,gemini_gmos_south


## Image type
    Classifies the images
    Adds image type columns to the fitstbl

In [18]:
filetypes = setupc.type_data(flag_unknown=True)

[INFO]    :: Typing files
[INFO]    :: Couldn't identify the following files:
[INFO]    :: S20180711S0183.fits
[INFO]    :: S20180711S0184.fits
[INFO]    :: S20180711S0185.fits
[INFO]    :: S20180711S0186.fits
[INFO]    :: S20180711S0259.fits
[INFO]    :: Typing completed!
[INFO]    :: Adding file type information to the fitstbl


### Show

In [19]:
setupc.fitstbl[['filename','arc','bias','pixelflat','science','standard','trace','unknown' ]]

filename,arc,bias,pixelflat,science,standard,trace,unknown
str19,bool,bool,bool,bool,bool,bool,bool
S20180711S0183.fits,False,False,False,False,False,False,True
S20180711S0184.fits,False,False,False,False,False,False,True
S20180711S0185.fits,False,False,False,False,False,False,True
S20180711S0186.fits,False,False,False,False,False,False,True
S20180711S0259.fits,False,False,False,False,False,False,True


# KLUDGING

In [20]:
setupc.fitstbl['unknown'] = False
setupc.fitstbl[0:2]['science'] = True
setupc.fitstbl[2]['pixelflat'] = True
setupc.fitstbl[2]['trace'] = True
setupc.fitstbl[3]['arc'] = True
setupc.fitstbl[-1]['standard'] = True
#setupc.fitstbl[-1]['science'] = False

#setupc.fitstbl[5]['pixelflat'] = True
#setupc.fitstbl[5]['trace'] = True
#
'''
setupc.fitstbl[-3]['standard'] = True
setupc.fitstbl[-3]['science'] = False
setupc.fitstbl[-2]['pixelflat'] = True
setupc.fitstbl[-2]['trace'] = True
setupc.fitstbl[-1]['arc'] = True
'''

"\nsetupc.fitstbl[-3]['standard'] = True\nsetupc.fitstbl[-3]['science'] = False\nsetupc.fitstbl[-2]['pixelflat'] = True\nsetupc.fitstbl[-2]['trace'] = True\nsetupc.fitstbl[-1]['arc'] = True\n"

In [21]:
setupc.fitstbl[['filename','arc','bias','pixelflat','science','standard','trace','unknown' ]]

filename,arc,bias,pixelflat,science,standard,trace,unknown
str19,bool,bool,bool,bool,bool,bool,bool
S20180711S0183.fits,False,False,False,True,False,False,False
S20180711S0184.fits,False,False,False,True,False,False,False
S20180711S0185.fits,False,False,True,False,False,True,False
S20180711S0186.fits,True,False,False,False,False,False,False
S20180711S0259.fits,False,False,False,False,True,False,False


## Match to science

In [22]:
spectrograph.calib_par['biasframe']['useframe'] = 'overscan'

In [23]:
fitstbl = setupc.match_to_science()

[INFO]    :: Matching calibrations to Science frames
[INFO]    :: =================================================
[INFO]    :: Matching calibrations to ESO-606-G036: S20180711S0183.fits
[INFO]    ::   Found 1 arc frame for ESO-606-G036 (1 required)
[INFO]    ::   Found 0 bias frame for ESO-606-G036 (5 required)
[WARNING] ::   Only 0/5 bias frames for ESO-606-G036
[INFO]    ::   Dark frames not required.  Not matching..
[INFO]    ::    No pinhole frames are required.  Not matching..
[INFO]    ::   Found 1 pixelflat frame for ESO-606-G036 (1 required)
[INFO]    ::   Found 1 standard frame for ESO-606-G036 (1 required)
[INFO]    ::   Found 1 trace frame for ESO-606-G036 (1 required)
[INFO]    :: =================================================
[INFO]    :: Matching calibrations to ESO-606-G036: S20180711S0184.fits
[INFO]    ::   Found 1 arc frame for ESO-606-G036 (1 required)
[INFO]    ::   Found 0 bias frame for ESO-606-G036 (5 required)
[WARNING] ::   Only 0/5 bias frames for ESO-606

### Setup dict

In [24]:
setup_dict = setupc.build_setup_dict()

In [25]:
setup_dict

{'A': {'--': {'dichroic': 'None',
   'disperser': {'angle': 'none', 'name': 'B600+_G5323'},
   'slit': {'decker': 'None', 'slitlen': 'none', 'slitwid': 'none'}},
  '01': {'binning': 'None', 'det': 1, 'namp': 4},
  '02': {'binning': 'None', 'det': 2, 'namp': 4},
  '03': {'binning': 'None', 'det': 3, 'namp': 4},
  'aa': {'arc': ['S20180711S0186.fits'],
   'bias': [],
   'pixelflat': ['S20180711S0185.fits'],
   'science': ['S20180711S0183.fits', 'S20180711S0184.fits'],
   'trace': ['S20180711S0185.fits']}}}

In [26]:
setupc.fitstbl[['filename','arc','bias','pixelflat','science','standard','sci_ID']]

filename,arc,bias,pixelflat,science,standard,sci_ID
str19,bool,bool,bool,bool,bool,int64
S20180711S0183.fits,False,False,False,True,False,1
S20180711S0184.fits,False,False,False,True,False,2
S20180711S0185.fits,False,False,True,False,False,3
S20180711S0186.fits,True,False,False,False,False,3
S20180711S0259.fits,False,False,False,False,True,3


----

## Setup + datasec

In [27]:
# Image IDs
sci_ID = 1  # First exposure ID
det = 1     # 
dnum = 'det01'

In [28]:
# Index in fitstbl
scidx = np.where((fitstbl['sci_ID'] == sci_ID) & fitstbl['science'])[0][0]
scidx

0

### Setup

In [29]:
setup = 'A_01_aa'

## Calibrations

In [30]:
reload(calibrations)
caliBrate = calibrations.MultiSlitCalibrations(fitstbl, save_masters=False, write_qa=False)

In [31]:
caliBrate.reset(setup, 1, sci_ID, spectrograph.calib_par)

## datasec_img

In [32]:
datasec_img = caliBrate.get_datasec_img()
datasec_img.shape

[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0183.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0183.fits


(1024, 2112)

## Bias

In [33]:
bias = caliBrate.get_bias()
bias

'overscan'

----

## Arc Image frame

In [34]:
arc = caliBrate.get_arc()

[INFO]    :: Preparing a master arc frame
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0186.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0186.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0186.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0186.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0186.fits
[INFO]    :: Bias subtracting your image(s)
[INFO]    :: Using overscan to subtact


In [35]:
caliBrate.show(arc)

----

## Bad pixel mask

In [36]:
bpm = caliBrate.get_bpm()

[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0183.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0183.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0183.fits


In [37]:
np.sum(bpm)

0

----

## pixlocn

In [38]:
pixlocn = caliBrate.get_pixlocn()
pixlocn.shape

[INFO]    :: Deriving physical pixel locations on the detector
[INFO]    :: Pixel gap in the dispersion direction = 0.000
[INFO]    :: Pixel size in the dispersion direction = 1.000
[INFO]    :: Pixel gap in the spatial direction = 0.000
[INFO]    :: Pixel size in the spatial direction = 1.000
[INFO]    :: Saving pixel locations


(1024, 2112, 4)

----

## Trace slit(s)

In [39]:
caliBrate.msbpm[:,0:37] = 1.
caliBrate.msbpm[:,-20:] = 1.

In [40]:
caliBrate.spectrograph.calib_par['slits']

    Parameter      Value    Default          Type  Callable
-----------------------------------------------------------
     function   legendre   legendre           str     False
    polyorder          3          3           int     False
       medrep          0          0           int     False
       number         -1         -1           int     False
         trim       3, 3       3, 3         tuple     False
       maxgap       None       None           int     False
     maxshift       0.15       0.15    int, float     False
          pad          0          0           int     False
    sigdetect       20.0       20.0    int, float     False
   fracignore       0.01       0.01         float     False
diffpolyorder          2          2           int     False
       single         []         []          list     False
   sobel_mode    nearest    nearest           str     False
          pca  see below  see below  ParSet, dict     False

pca
  Parameter             Value      

In [41]:
caliBrate.par['slits']['sigdetect'] = 300.
caliBrate.par['slits']['pca']['params'] = [1,0]

In [42]:
tslits_dict, maskslits = caliBrate.get_slits()

[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0185.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0185.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0185.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0185.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0185.fits
[INFO]    :: Bias subtracting your image(s)
[INFO]    :: Using overscan to subtact
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0185.fits
[INFO]    :: Reading GMOS file: /data/Projects/Python/PYPIT-development-suite/RAW_DATA/Gemini_GMOS/B600/S20180711S0185.fits
[WARNING] :: Should probably be measuring the gai

In [43]:
caliBrate.show(caliBrate.traceSlits.mstrace)

In [44]:
caliBrate.traceSlits.show('siglev')

In [45]:
caliBrate.traceSlits.show('edges')

### Mask the uninteresting slits

In [ ]:
caliBrate.maskslits[0:2]= True
caliBrate.maskslits[-1]= True
caliBrate.maskslits

array([ True,  True, False,  True])

----

## Wavelength Calibration

In [ ]:
caliBrate.get_wv_calib()

[INFO]    :: Extracting an approximate arc spectrum at the centre of each slit
[WARNING] :: Unable to parse input binning: None
[WARNING] :: Assuming unbinned, i.e.  1x1
[INFO]    :: Loading line list using CuI,ArI lamps
[INFO]    :: Rejecting select ArI lines
[WARNING] :: Am not sure this method does anything for real -- JXP 03-Jul-2018
[INFO]    :: Cutting down line list by wvmnx: 3800,8000


> /data/Projects/Python/arclines/arclines/io.py(147)load_line_lists()
-> raise IOError("Input line {:s} is not included in arclines".format(line))
(Pdb) line_file
'/data/Projects/Python/arclines/arclines/data/lists/CuI_lines.dat'
(Pdb) NIST
False
(Pdb) w
  /home/xavier/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/runpy.py(193)_run_module_as_main()
-> "__main__", mod_spec)
  /home/xavier/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/runpy.py(85)_run_code()
-> exec(code, run_globals)
  /home/xavier/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/ipykernel_launcher.py(16)<module>()
-> app.launch_new_instance()
  /home/xavier/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/traitlets/config/application.py(658)launch_instance()
-> app.start()
  /home/xavier/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/ipykernel/kernelapp.py(477)start()
-> ioloop.IOLoop.instance().start()
  /home/xavier/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/zmq/eve

In [ ]:
fitstbl

----

## Wave Tilts

In [ ]:
# Settings kludges
tilt_settings = dict(tilts=settings.argflag['trace']['slits']['tilts'].copy(),
                     masters=settings.argflag['reduce']['masters'])
tilt_settings['tilts']['function'] = settings.argflag['trace']['slits']['function']

In [ ]:
# Instantiate
waveTilts = wavetilts.WaveTilts(msarc, settings=tilt_settings,
                                    det=det, setup=setup,
                                    tslits_dict=tslits_dict, settings_det=settings_det,
                                    pixlocn=pixlocn)

In [ ]:
# Run
mstilts, wt_maskslits = waveTilts.run(maskslits=maskslits, wv_calib=wv_calib)

In [ ]:
waveTilts.show('fweight', slit=0)

In [ ]:
waveTilts.show('tilts', slit=0)

----

## Pixel Flat Field

In [ ]:
# Settings
flat_settings = dict(flatfield=settings.argflag['reduce']['flatfield'].copy(),
                     slitprofile=settings.argflag['reduce']['slitprofile'].copy(),
                     combine=settings.argflag['pixelflat']['combine'].copy(),
                     masters=settings.argflag['reduce']['masters'].copy(),
                     detector=settings.spect[dnum])

In [ ]:
# Instantiate
pixflat_image_files = arsort.list_of_files(fitstbl, 'pixelflat', sci_ID)
flatField = flatfield.FlatField(file_list=pixflat_image_files, msbias=msbias,
                                spectrograph=spectrograph,
                                settings=flat_settings,
                                tslits_dict=tslits_dict,
                                tilts=mstilts, det=det, setup=setup,
                                datasec_img=datasec_img)

In [ ]:
# Run
mspixflatnrm, slitprof = flatField.run(armed=False)

In [ ]:
flatField.show('norm')

----

## Wavelength Image

In [ ]:
# Settings
wvimg_settings = dict(masters=settings.argflag['reduce']['masters'].copy())

In [ ]:
# Instantiate
waveImage = waveimage.WaveImage(mstilts, wv_calib, settings=wvimg_settings,
                                    setup=setup, maskslits=maskslits,
                                    slitpix=tslits_dict['slitpix'])

In [ ]:
# Build
mswave = waveImage._build_wave()

In [ ]:
waveImage.show('wave')

----

## Science Image

### File list

In [ ]:
sci_image_files = arsort.list_of_files(fitstbl, 'science', sci_ID)

In [ ]:
# Settings
sci_settings = tsettings.copy()

### Instantiate

In [ ]:
# Instantiate
sciI = scienceimage.ScienceImage(file_list=sci_image_files, datasec_img=datasec_img,
                                 bpm=msbpm, det=det, setup=setup, settings=sci_settings,
                                 maskslits=maskslits, pixlocn=pixlocn, tslits_dict=tslits_dict,
                                 tilts=mstilts, fitstbl=fitstbl, scidx=scidx)

### Name, time

In [ ]:
# Names and time
obstime, basename = sciI.init_time_names(settings.spect['mosaic']['camera'],
                timeunit=settings.spect["fits"]["timeunit"])
basename

### Process

In [ ]:
# Process (includes Variance image and CRs)
dnoise = (settings_det['darkcurr'] * float(fitstbl["exptime"][scidx])/3600.0)
sciframe, rawvarframe, crmask = sciI._process(
    msbias, mspixflatnrm, apply_gain=True, dnoise=dnoise)

In [ ]:
sciI.show('sci')

### Global sky sub

In [ ]:
# Global skysub
settings_skysub = {}
settings_skysub['skysub'] = settings.argflag['reduce']['skysub'].copy()
global_sky, modelvarframe = sciI.global_skysub(settings_skysub)

In [ ]:
sciI.show('skysub')

### Find objects

In [ ]:
_, nobj = sciI.find_objects()

### Repeat the last 2 steps

In [ ]:
# Mask the objects
global_sky, modelvarframe = sciI.global_skysub(settings_skysub, use_tracemask=True)
# Another round of finding objects
_, nobj = sciI.find_objects()  

### Extraction -- New algorithm in development

In [ ]:
specobjs, finalvar, finalsky = sciI.extraction(mswave)

### Flexure

In [ ]:
flex_list = arwave.flexure_obj(
    specobjs, maskslits, settings.argflag['reduce']['flexure']['method'],
    spectrograph,
    skyspec_fil = settings.argflag['reduce']['flexure']['spectrum'],
    mxshft = settings.argflag['reduce']['flexure']['maxshift'])

In [ ]:
# QA 
arwave.flexure_qa(specobjs, maskslits, basename, det, flex_list)

### Heliocentric (optional)

In [ ]:
vel, vel_corr = arwave.geomotion_correct(specobjs, maskslits, fitstbl, scidx,
                                         obstime, settings.spect,
                                         settings.argflag['reduce']['calibrate']['refframe'])

In [ ]:
sci_dict = {}
sci_dict['meta'] = {}
sci_dict['meta']['vel_corr'] = vel_corr

----

## Write

### 1D spectra

In [ ]:
outfile = 'Science/spec1d_{:s}.fits'.format(basename)
helio_dict = dict(refframe=settings.argflag['reduce']['calibrate']['refframe'],
                  vel_correction=sci_dict['meta']['vel_corr'])
arsave.save_1d_spectra_fits([specobjs], fitstbl[scidx], outfile,
                                helio_dict=helio_dict, obs_dict=settings.spect['mosaic'])

### 2D images

In [ ]:
# Write 2D images for the Science Frame
arsave.save_2d_images(
    sci_dict, fitstbl, scidx,
    settings.spect['fits']['headext{0:02d}'.format(1)], setup,
    settings.argflag['run']['directory']['master']+'_'+spectrograph, # MFDIR
    'Science/',  basename)

----

## Fluxing (optional)

### Reduce a standard star

In [ ]:
std_dict = {}
# Reduce standard here; only legit if the mask is the same
std_idx = arsort.ftype_indices(fitstbl, 'standard', sci_ID)[0]
#
std_image_files = arsort.list_of_files(fitstbl, 'standard', sci_ID)
std_dict[std_idx] = {}

# Instantiate for the Standard
stdI = scienceimage.ScienceImage(file_list=std_image_files, datasec_img=datasec_img,
                                 bpm=msbpm, det=det, setup=setup, settings=sci_settings,
                                 maskslits=maskslits, pixlocn=pixlocn, tslits_dict=tslits_dict,
                                 tilts=mstilts, fitstbl=fitstbl, scidx=std_idx,
                                 objtype='standard')
# Names and time
_, std_basename = stdI.init_time_names(settings.spect['mosaic']['camera'],
                                         timeunit=settings.spect["fits"]["timeunit"])
# Process (includes Variance image and CRs)
stdframe, _, _ = stdI._process(msbias, mspixflatnrm, apply_gain=True, dnoise=dnoise)
# Sky
_ = stdI.global_skysub(settings_skysub)
# Find objects
_, nobj = stdI.find_objects()
_ = stdI.global_skysub(settings_skysub, use_tracemask=True)
# Extract
stdobjs, _, _ = stdI.extraction(mswave)
# Save for fluxing and output later
std_dict[std_idx][det] = {}
std_dict[std_idx][det]['basename'] = std_basename
std_dict[std_idx][det]['specobjs'] = arutils.unravel_specobjs([stdobjs])


### Sensitivity function

In [ ]:
# Settings
fsettings = settings.spect.copy()
fsettings['run'] = settings.argflag['run']
fsettings['reduce'] = settings.argflag['reduce']

In [ ]:
# Build the list of stdobjs
reload(fluxspec)
all_std_objs = []
for det in std_dict[std_idx].keys():
    all_std_objs += std_dict[std_idx][det]['specobjs']
FxSpec = fluxspec.FluxSpec(settings=fsettings, std_specobjs=all_std_objs,
                           setup=setup)  # This takes the last setup run, which is as sensible as any..
sensfunc = FxSpec.master(fitstbl[std_idx], save=False)

In [ ]:
all_std_objs

In [ ]:
# Show
FxSpec.show_sensfunc()

### Flux

In [ ]:
# Load
sci_specobjs, sci_header = arload.load_specobj('Science/spec1d_OFF_J1044p6306_LRISr_2016Feb16T112439.fits')
#
FxSpec.sci_specobjs = sci_specobjs
FxSpec.sci_header = sci_header
# Flux
FxSpec.flux_science()

In [ ]:
# Write
FxSpec.write_science('Science/spec1d_OFF_J1044p6306_LRISr_2016Feb16T112439.fits')